In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
df_recipes2 = pd.read_parquet('../Data/recipes.parquet')
df_recipes2.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."


In [4]:
df_recipes2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   RecipeId                    522517 non-null  float64            
 1   Name                        522517 non-null  object             
 2   AuthorId                    522517 non-null  int32              
 3   AuthorName                  522517 non-null  object             
 4   CookTime                    439972 non-null  object             
 5   PrepTime                    522517 non-null  object             
 6   TotalTime                   522517 non-null  object             
 7   DatePublished               522517 non-null  datetime64[ns, UTC]
 8   Description                 522512 non-null  object             
 9   Images                      522516 non-null  object             
 10  RecipeCategory              521766 non-null 

In [5]:
# Drop unnecessary columns
columns_to_drop = ['RecipeId', 'AuthorId', 'AuthorName', 'DatePublished', 'Description','RecipeYield']
df_recipes2 = df_recipes2.drop(columns=columns_to_drop)

In [6]:
df_2M = pd.read_csv('../Data/recipes_data.csv')

In [7]:
df_2M

,title,ingredients,directions,link,source,NER,site
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""bite size shredded rice biscuits"", ""vanilla""...",www.cookbooks.com
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""cream of mushroom soup"", ""beef"", ""sour cream...",www.cookbooks.com
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""pepper"", ""cream cheese"", ""gar...",www.cookbooks.com
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken gravy"", ""cream of mushroom soup"", ""c...",www.cookbooks.com
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""graham cracker crumbs"", ""powdered sugar"", ""p...",www.cookbooks.com
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""marshmallows"", ...",www.foodnetwork.com
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""choice"", ""miracle whip"", ""eggs"", ""relish"", ""...",cookpad.com
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""soy sauce"", ""radish"", ""white sesame seeds"", ...",cookpad.com
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""egg"", ""sugar"", ""freshly groun...",cooking.nytimes.com


### Merge the two datasets to get units of measurement

In [8]:
df = pd.merge(df_recipes2, df_2M, left_on='Name', right_on='title', how='left')

In [9]:
df.head()

,Name,CookTime,PrepTime,TotalTime,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,...,ProteinContent,RecipeServings,RecipeInstructions,title,ingredients,directions,link,source,NER,site
0,Low-Fat Berry Blue Frozen Dessert,PT24H,PT45M,PT24H45M,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,"[Dessert, Low Protein, Low Cholesterol, Health...","[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",4.5,...,3.2,4.0,"[Toss 2 cups berries with sugar., Let stand fo...",Low-Fat Berry Blue Frozen Dessert,"[""4 cups blueberries, fresh or frozen"", ""1/4 c...","[""Toss 2 cups berries with sugar."", ""Let stand...",www.food.com/recipe/low-fat-berry-blue-frozen-...,Gathered,"[""blueberries"", ""sugar"", ""vanilla yogurt"", ""le...",www.food.com
1,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1/2 lb. meat or chicken, cut in pieces"", ""1/...","[""Marinate meat or chicken in thick yogurt for...",www.cookbooks.com/Recipe-Details.aspx?id=548876,Gathered,"[""meat"", ""ginger"", ""cinnamon"", ""cloves"", ""onio...",www.cookbooks.com
2,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""1 tablespoon saffron"", ""4 teaspoons milk, wa...","[""Soak saffron in warm milk for 5 minutes and ...",www.food.com/recipe/biryani-39,Gathered,"[""tomatoes"", ""basmati rice"", ""coriander seed"",...",www.food.com
3,Biryani,PT25M,PT4H,PT4H25M,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,"[Chicken Thigh & Leg, Chicken, Poultry, Meat, ...","[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",3.0,...,63.4,6.0,[Soak saffron in warm milk for 5 minutes and p...,Biryani,"[""2 cups basmati rice"", ""34 kg chicken piece"",...","[""Mix tomato puree, yogurt, ginger garlic past...",www.food.com/recipe/biryani-484928,Recipes1M,"[""oil"", ""basmati rice"", ""cumin powder"", ""tomat...",www.food.com
4,Best Lemonade,PT5M,PT30M,PT35M,[https://img.sndimg.com/food/image/upload/w_55...,Beverages,"[Low Protein, Low Cholesterol, Healthy, Summer...","[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...",4.5,...,0.3,4.0,"[Into a 1 quart Jar with tight fitting lid, pu...",Best Lemonade,"[""1 1/2 c. sugar"", ""1 Tbsp. lemon peel, finely...","[""In 1-quart jar with tight fitting lid, shake...",www.cookbooks.com/Recipe-Details.aspx?id=293155,Gathered,"[""sugar"", ""lemon juice"", ""cold water"", ""very h...",www.cookbooks.com


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11832728 entries, 0 to 11832727
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Name                        object 
 1   CookTime                    object 
 2   PrepTime                    object 
 3   TotalTime                   object 
 4   Images                      object 
 5   RecipeCategory              object 
 6   Keywords                    object 
 7   RecipeIngredientQuantities  object 
 8   RecipeIngredientParts       object 
 9   AggregatedRating            float64
 10  ReviewCount                 float64
 11  Calories                    float64
 12  FatContent                  float64
 13  SaturatedFatContent         float64
 14  CholesterolContent          float64
 15  SodiumContent               float64
 16  CarbohydrateContent         float64
 17  FiberContent                float64
 18  SugarContent                float64
 19  ProteinContent     

### Remove duplicate rows

### Convert cooktime to minutes

In [13]:
def time_to_minutes(time):
    if pd.isna(time): 
        return np.nan

    hours = re.search(r'(\d+)H', time)
    minutes = re.search(r'(\d+)M', time)
    
    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))
    
    return total_minutes

In [14]:
def time_to_readable_format(time):
    if pd.isna(time): 
        return np.nan

    hours = re.search(r'(\d+)H', time)
    minutes = re.search(r'(\d+)M', time)
    
    result = []
    if hours:
        result.append(f"{int(hours.group(1))} hour{'s' if int(hours.group(1)) > 1 else ''}")
    if minutes:
        result.append(f"{int(minutes.group(1))} minute{'s' if int(minutes.group(1)) > 1 else ''}")
    
    return ' '.join(result)

In [15]:
df['CookTime'].apply(time_to_readable_format)

0             24 hours
1           25 minutes
2           25 minutes
3           25 minutes
4            5 minutes
               ...    
11832723    35 minutes
11832724       3 hours
11832725       3 hours
11832726           NaN
11832727    25 minutes
Name: CookTime, Length: 11832728, dtype: object

In [16]:
columns = ['CookTime', 'PrepTime', 'TotalTime']
for col in columns:
    df[f'{col}_minutes'] = df[col].apply(time_to_minutes)

### Column Images : only keep the first link

In [17]:
def extract_first_element(x):
    if isinstance(x, np.ndarray): 
        return x[0] if x.size > 0 else None  
    elif pd.notna(x):  
        return x  
    else:
        return None

In [18]:
df['Images'] = df['Images'].apply(extract_first_element)

In [7]:
df_recipes2['Images'][0]

array(['https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/YUeirxMLQaeE1h3v3qnM_229%20berry%20blue%20frzn%20dess.jpg',
       'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/AFPDDHATWzQ0b1CDpDAT_255%20berry%20blue%20frzn%20dess.jpg',
       'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/UYgf9nwMT2SGGJCuzILO_228%20berry%20blue%20frzn%20dess.jpg',
       'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/PeBMJN2TGSaYks2759BA_20140722_202142.jpg',
       'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/picuaETeN.jpg',
       'https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pictzvxW5.jpg'],
      dtype=object)